In [2]:
# Install libraries
import pandas as pd
import html5lib
!pip install lxml 
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

dfs = pd.read_html(wiki_url)

     |████████████████████████████████| 5.5MB 4.2MB/s eta 0:00:01


In [3]:
df = dfs[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
# Clean up data. Remove codes where Borough is Not assigned. Save original 
# data, just in case we need it in the future. We also reset index and drop old 
# index to keep new dataframe clean.
new_df = df
new_df = new_df[df.Borough != 'Not assigned'].reset_index(drop=True)
new_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
# Change Neighbourhood to Borough name if Neighbourhood is Not assigned. For this,
# change Neighbourhood string to a list so that each item of the list can be checked
# and name of item can be changed if needed.
for rowNum in range(len(new_df)):
    neigh_list = new_df.iloc[rowNum]['Neighbourhood'].split(",")
    neigh_list = [item.strip() for item in neigh_list]
    new_df.iloc[rowNum]['Neighbourhood'] = neigh_list
    for neighNum in range(len(neigh_list)):
        if neigh_list[neighNum] == 'Not assigned':
            new_df.iloc[rowNum]['Neighbourhood'][neighNum] = new_df.iloc[rowNum]['Borough']

In [7]:
new_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront]"
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]"
4,M7A,Downtown Toronto,"[Queen's Park, Ontario Provincial Government]"


In [5]:
new_df.shape

(103, 3)

In [8]:
# Geo data for latitude and longitude
!wget -q -O 'toronto_geo.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [10]:
# Create dataframe of postal codes with their latitudes and longitudes
toronto_geo_df = pd.read_csv('toronto_geo.csv')
toronto_geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
# Join the 2 dataframes to add latitude and longitude information for all
# Toronto Boroughs
toronto_df = new_df.join(toronto_geo_df.set_index('Postal Code'), on='Postal Code')
toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],43.753259,-79.329656
1,M4A,North York,[Victoria Village],43.725882,-79.315572
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront]",43.654260,-79.360636
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]",43.718518,-79.464763
4,M7A,Downtown Toronto,"[Queen's Park, Ontario Provincial Government]",43.662301,-79.389494
